In [62]:
import pandas as pd 
dados_viagem = "/home/joao/Estudos_py_ml/analise_de_dados/projeto_pandas_governo/2023_Viagem.csv"

df_viagens = pd.read_csv(dados_viagem, encoding="Windows-1252", sep=";", decimal=",")
pd.set_option('display.max_columns', None)



In [63]:
df_viagens

,Identificador do processo de viagem,Número da Proposta (PCDP),Situação,Viagem Urgente,Justificativa Urgência Viagem,Código do órgão superior,Nome do órgão superior,Código órgão solicitante,Nome órgão solicitante,CPF viajante,Nome,Cargo,Função,Descrição Função,Período - Data de início,Período - Data de fim,Destinos,Motivo,Valor diárias,Valor passagens,Valor devolução,Valor outros gastos
0,17821923,000001/23-1C,Realizada,NÃO,Sem informação,26000,Ministério da Educação,26352,Fundação Universidade Federal do ABC,***.875.238-**,PRISCILA LEAL DA SILVA,PROFESSOR DO MAGISTERIO SUPERIOR,-1,Sem informação,01/01/2023,02/12/2023,Loughborough/Reino Unido,Intercâmbio acadêmico PCDP anterior 07/21-3C,0.00,0.00,0.00,0.00
1,18236583,000018/23,Realizada,NÃO,Sem informação,26000,Ministério da Educação,26236,Universidade Federal Fluminense,NaN,CAMILLA DUARTE DA SILVA,BIBLIOTECARIO-DOCUMENTALISTA,-1,Sem informação,16/01/2023,28/02/2024,Rio de Janeiro/RJ,Afastamento para pós-graduação stricto sensu n...,0.00,0.00,0.00,0.00
2,18288418,000007/23-1C,Realizada,SIM,Por necessidade do serviço.,52000,Ministério da Defesa,52121,Comando do Exército,***.621.358-**,ANDRE LUIS COSTA PITANGUEIRA,NaN,OfSuperior,OfSuperior,10/01/2023,29/01/2023,Nova York/Estados Unidos da América,Participar da revisão do COE Manual (Contingen...,39565.70,7434.13,0.00,1095.35
3,18296348,000070/23,Realizada,SIM,A efetivação do empenho no Siafi e a respectiv...,26000,Ministério da Educação,26406,Instituto Federal do Espírito Santo,***.407.547-**,LIDIANY MIRANDA FERRAZ NUNES,AUXILIAR DE BIBLIIOTECA,-1,Sem informação,05/02/2023,10/02/2023,Vitória/ES,Participação das aulas de Mestrado Interinstit...,1013.96,0.00,0.00,0.00
4,18302983,000001/23,Realizada,NÃO,Sem informação,52000,Ministério da Defesa,52111,Comando da Aeronáutica,***.855.388-**,GREGOR GASPAR,NaN,OfIntermed,OfIntermed,01/01/2023,07/07/2023,Gotemburgo/Suécia,PARTICIPAR DO CURSO DE PILOTO DE GRIPEN C/D.,0.00,14903.10,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
826984,2023001618,Sem informaçã,Realizada,NÃO,Sem informação,-1,Sem informação,-1,Sem informação,***.748.269-**,BIANCA VIANA KIVEL,ANALISTA DO BANCO CENTRAL,-1,Sem informação,05/12/2023,06/12/2023,Sem informação,Participar de tratativas nos locais das reuniõ...,606.89,3331.29,0.00,0.00
826985,2023001619,Sem informaçã,Realizada,NÃO,Sem informação,-1,Sem informação,-1,Sem informação,***.231.806-**,LUCAS ALVES FREIRE,PROCURADOR DO BANCO CENTRAL,-1,Sem informação,11/12/2023,13/12/2023,Sem informação,Participar da reunião do COPOM .,1089.00,1837.18,0.00,0.00
826986,2023001628,Sem informaçã,Realizada,NÃO,Sem informação,-1,Sem informação,-1,Sem informação,***.632.011-**,SERGIO BOTELHO,TECNICO DO BANCO CENTRAL,-1,Sem informação,07/12/2023,08/12/2023,Sem informação,Acompanhamento do Presidente do BC - São Paulo,734.13,4555.95,0.00,0.00
826987,2023001631,Sem informaçã,Realizada,NÃO,Sem informação,-1,Sem informação,-1,Sem informação,***.407.445-**,NELSON CAMPOS REIS JUNIOR,TECNICO DO BANCO CENTRAL,-1,Sem informação,11/12/2023,11/12/2023,Sem informação,Acompanhamento do Presidente - São Paulo.,319.28,3745.53,0.00,400.00


In [64]:
df_viagens["Nome do órgão superior"]


0         Ministério da Educação
1         Ministério da Educação
2           Ministério da Defesa
3         Ministério da Educação
4           Ministério da Defesa
                   ...          
826984            Sem informação
826985            Sem informação
826986            Sem informação
826987            Sem informação
826988            Sem informação
Name: Nome do órgão superior, Length: 826989, dtype: object

In [66]:
df_viagens["Valor diárias"]

0            0.00
1            0.00
2        39565.70
3         1013.96
4            0.00
           ...   
826984     606.89
826985    1089.00
826986     734.13
826987     319.28
826988       0.00
Name: Valor diárias, Length: 826989, dtype: float64

In [67]:
##para mostrar dois elementos , como valor da diaria e nome do orgao voce cria uma lista com esses dois elementos 

colunas = ["Nome do órgão superior","Valor diárias"]
df_viagens[colunas]


,Nome do órgão superior,Valor diárias
0,Ministério da Educação,0.00
1,Ministério da Educação,0.00
2,Ministério da Defesa,39565.70
3,Ministério da Educação,1013.96
4,Ministério da Defesa,0.00
...,...,...
826984,Sem informação,606.89
826985,Sem informação,1089.00
826986,Sem informação,734.13
826987,Sem informação,319.28


In [68]:
##Usando metodo upper para botar as letras para maiusculo e usando o replace para abreviar para MIN

df_viagens["Nome do órgão superior"].str.upper().str.replace("MINISTÉRIO", "MIN.")


0         MIN. DA EDUCAÇÃO
1         MIN. DA EDUCAÇÃO
2           MIN. DA DEFESA
3         MIN. DA EDUCAÇÃO
4           MIN. DA DEFESA
                ...       
826984      SEM INFORMAÇÃO
826985      SEM INFORMAÇÃO
826986      SEM INFORMAÇÃO
826987      SEM INFORMAÇÃO
826988      SEM INFORMAÇÃO
Name: Nome do órgão superior, Length: 826989, dtype: object

In [69]:
df_viagens[["Valor diárias", "Valor passagens"]]

,Valor diárias,Valor passagens
0,0.00,0.00
1,0.00,0.00
2,39565.70,7434.13
3,1013.96,0.00
4,0.00,14903.10
...,...,...
826984,606.89,3331.29
826985,1089.00,1837.18
826986,734.13,4555.95
826987,319.28,3745.53


In [72]:
##mudando o tipo de str para float

df_viagens['Valor diárias'] = df_viagens['Valor diárias']


In [32]:
df_viagens["Valor diárias"]

0            0.00
1            0.00
2        39565.70
3         1013.96
4            0.00
           ...   
826984     606.89
826985    1089.00
826986     734.13
826987     319.28
826988       0.00
Name: Valor diárias, Length: 826989, dtype: float64

In [73]:
df_viagens['Valor outros gastos'] = df_viagens['Valor outros gastos']
df_viagens['Valor passagens'] = df_viagens['Valor passagens']
df_viagens["Valor devolução"] = df_viagens['Valor devolução']

In [74]:
df_viagens.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 826989 entries, 0 to 826988
Data columns (total 22 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   Identificador do processo de viagem  826989 non-null  int64  
 1   Número da Proposta (PCDP)            826989 non-null  object 
 2   Situação                             826989 non-null  object 
 3   Viagem Urgente                       826989 non-null  object 
 4   Justificativa Urgência Viagem        826887 non-null  object 
 5   Código do órgão superior             826989 non-null  int64  
 6   Nome do órgão superior               826989 non-null  object 
 7   Código órgão solicitante             826989 non-null  int64  
 8   Nome órgão solicitante               826989 non-null  object 
 9   CPF viajante                         748302 non-null  object 
 10  Nome                                 826989 non-null  object 
 11  Cargo        

In [75]:
df_viagens['Valor diárias'] + df_viagens['Valor passagens']




0            0.00
1            0.00
2        46999.83
3         1013.96
4        14903.10
           ...   
826984    3938.18
826985    2926.18
826986    5290.08
826987    4064.81
826988    2961.55
Length: 826989, dtype: float64

In [76]:
df_viagens["Despesas"] = df_viagens["Valor diárias"] + df_viagens["Valor passagens"] + df_viagens["Valor outros gastos"]

In [77]:
df_viagens

,Identificador do processo de viagem,Número da Proposta (PCDP),Situação,Viagem Urgente,Justificativa Urgência Viagem,Código do órgão superior,Nome do órgão superior,Código órgão solicitante,Nome órgão solicitante,CPF viajante,Nome,Cargo,Função,Descrição Função,Período - Data de início,Período - Data de fim,Destinos,Motivo,Valor diárias,Valor passagens,Valor devolução,Valor outros gastos,Despesas
0,17821923,000001/23-1C,Realizada,NÃO,Sem informação,26000,Ministério da Educação,26352,Fundação Universidade Federal do ABC,***.875.238-**,PRISCILA LEAL DA SILVA,PROFESSOR DO MAGISTERIO SUPERIOR,-1,Sem informação,01/01/2023,02/12/2023,Loughborough/Reino Unido,Intercâmbio acadêmico PCDP anterior 07/21-3C,0.00,0.00,0.00,0.00,0.00
1,18236583,000018/23,Realizada,NÃO,Sem informação,26000,Ministério da Educação,26236,Universidade Federal Fluminense,NaN,CAMILLA DUARTE DA SILVA,BIBLIOTECARIO-DOCUMENTALISTA,-1,Sem informação,16/01/2023,28/02/2024,Rio de Janeiro/RJ,Afastamento para pós-graduação stricto sensu n...,0.00,0.00,0.00,0.00,0.00
2,18288418,000007/23-1C,Realizada,SIM,Por necessidade do serviço.,52000,Ministério da Defesa,52121,Comando do Exército,***.621.358-**,ANDRE LUIS COSTA PITANGUEIRA,NaN,OfSuperior,OfSuperior,10/01/2023,29/01/2023,Nova York/Estados Unidos da América,Participar da revisão do COE Manual (Contingen...,39565.70,7434.13,0.00,1095.35,48095.18
3,18296348,000070/23,Realizada,SIM,A efetivação do empenho no Siafi e a respectiv...,26000,Ministério da Educação,26406,Instituto Federal do Espírito Santo,***.407.547-**,LIDIANY MIRANDA FERRAZ NUNES,AUXILIAR DE BIBLIIOTECA,-1,Sem informação,05/02/2023,10/02/2023,Vitória/ES,Participação das aulas de Mestrado Interinstit...,1013.96,0.00,0.00,0.00,1013.96
4,18302983,000001/23,Realizada,NÃO,Sem informação,52000,Ministério da Defesa,52111,Comando da Aeronáutica,***.855.388-**,GREGOR GASPAR,NaN,OfIntermed,OfIntermed,01/01/2023,07/07/2023,Gotemburgo/Suécia,PARTICIPAR DO CURSO DE PILOTO DE GRIPEN C/D.,0.00,14903.10,0.00,0.00,14903.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
826984,2023001618,Sem informaçã,Realizada,NÃO,Sem informação,-1,Sem informação,-1,Sem informação,***.748.269-**,BIANCA VIANA KIVEL,ANALISTA DO BANCO CENTRAL,-1,Sem informação,05/12/2023,06/12/2023,Sem informação,Participar de tratativas nos locais das reuniõ...,606.89,3331.29,0.00,0.00,3938.18
826985,2023001619,Sem informaçã,Realizada,NÃO,Sem informação,-1,Sem informação,-1,Sem informação,***.231.806-**,LUCAS ALVES FREIRE,PROCURADOR DO BANCO CENTRAL,-1,Sem informação,11/12/2023,13/12/2023,Sem informação,Participar da reunião do COPOM .,1089.00,1837.18,0.00,0.00,2926.18
826986,2023001628,Sem informaçã,Realizada,NÃO,Sem informação,-1,Sem informação,-1,Sem informação,***.632.011-**,SERGIO BOTELHO,TECNICO DO BANCO CENTRAL,-1,Sem informação,07/12/2023,08/12/2023,Sem informação,Acompanhamento do Presidente do BC - São Paulo,734.13,4555.95,0.00,0.00,5290.08
826987,2023001631,Sem informaçã,Realizada,NÃO,Sem informação,-1,Sem informação,-1,Sem informação,***.407.445-**,NELSON CAMPOS REIS JUNIOR,TECNICO DO BANCO CENTRAL,-1,Sem informação,11/12/2023,11/12/2023,Sem informação,Acompanhamento do Presidente - São Paulo.,319.28,3745.53,0.00,400.00,4464.81


In [78]:
pd.set_option('display.float_format','{:.2f}'.format)
(df_viagens["Cargo"].value_counts(normalize=True)*100).rename("Proporção de Viagens").reset_index()

,Cargo,Proporção de Viagens
0,Informações protegidas por sigilo,23.40
1,PROFESSOR DO MAGISTERIO SUPERIOR,10.80
2,PROFESSOR ENS BASICO TECN TECNOLOGICO,7.42
3,CONTRATADO LEI 8745/93 - NI,3.52
4,AUDITOR FISCAL FEDERAL AGROPECUARIO,3.28
...,...,...
816,TECNICO EM HIGIENE DENTAL,0.00
817,OPERADOR DE TELE IMPRESSORA,0.00
818,ASSISTENTE TECNICO EM DESENV REGIONAL,0.00
819,TECNICO DE MANUTENCAO,0.00


In [81]:
##depesas por agrumento de Cargo 
pd.set_option('display.float_format','{:.2f}'.format)
df_viagens.groupby("Cargo")["Despesas"].sum().reset_index().sort_values(by="Despesas",ascending=False)

,Cargo,Despesas
443,Informações protegidas por sigilo,370919369.30
568,PROFESSOR DO MAGISTERIO SUPERIOR,110890764.22
569,PROFESSOR ENS BASICO TECN TECNOLOGICO,52548741.25
685,TECNICO DO SEGURO SOCIAL,45881679.92
72,ANALISTA AMBIENTAL,35344371.79
...,...,...
152,ASG - OPERADOR DE MAQUINA E EQUIPAMENTOS,0.00
733,TECNICO EM METALURGIA,0.00
817,VIDREIRO,0.00
220,AUXILIAR - AUXILIAR DE SERVICOS GERAIS,0.00


In [82]:
viagens_por_cargo = (df_viagens["Cargo"].value_counts(normalize=True)*100).rename("Proporção de Viagens").reset_index()

In [84]:
filtro_mais_de_1_pct = viagens_por_cargo["Proporção de Viagens"] > 1 

In [85]:
viagens_por_cargo

,Cargo,Proporção de Viagens
0,Informações protegidas por sigilo,23.40
1,PROFESSOR DO MAGISTERIO SUPERIOR,10.80
2,PROFESSOR ENS BASICO TECN TECNOLOGICO,7.42
3,CONTRATADO LEI 8745/93 - NI,3.52
4,AUDITOR FISCAL FEDERAL AGROPECUARIO,3.28
...,...,...
816,TECNICO EM HIGIENE DENTAL,0.00
817,OPERADOR DE TELE IMPRESSORA,0.00
818,ASSISTENTE TECNICO EM DESENV REGIONAL,0.00
819,TECNICO DE MANUTENCAO,0.00


In [45]:
viagens_por_cargo[filtro_mais_de_1_pct]

,Cargo,Proporção de Viagens
0,Informações protegidas por sigilo,23.40
1,PROFESSOR DO MAGISTERIO SUPERIOR,10.80
2,PROFESSOR ENS BASICO TECN TECNOLOGICO,7.42
3,CONTRATADO LEI 8745/93 - NI,3.52
4,AUDITOR FISCAL FEDERAL AGROPECUARIO,3.28
5,ANALISTA AMBIENTAL,2.69
6,ASSISTENTE EM ADMINISTRAçãO,2.31
7,TECNICO INFOR GEOGRAFICAS E ESTATISTICA,2.15
8,TECNICO DO SEGURO SOCIAL,2.11
9,AUDITOR-FISCAL DA RECEITA FEDERAL BRASIL,2.00


In [87]:
gasto_totais_por_cargo = df_viagens.groupby("Cargo")["Despesas"].sum().reset_index()

In [88]:
gasto_totais_por_cargo[gasto_totais_por_cargo['Despesas']> 10_000_000]

,Cargo,Despesas
15,AGENTE ADMINISTRATIVO,15010847.19
72,ANALISTA AMBIENTAL,35344371.79
95,ANALISTA DO SEGURO SOCIAL,15988307.12
113,ANALISTA TRIBUTARIO REC FEDERAL BRASIL,14868670.22
179,ASSISTENTE EM ADMINISTRAçãO,15094703.63
207,AUDITOR FISCAL DO TRABALHO,10376688.67
208,AUDITOR FISCAL FEDERAL AGROPECUARIO,24306782.30
209,AUDITOR-FISCAL DA RECEITA FEDERAL BRASIL,24405946.47
315,CONTRATADO LEI 8745/93 - NI,17510463.07
385,ESP EM REG DE SERV PUB DE TELECOMUNIC,10037621.59


In [89]:
df_viagens['Cargo'] = df_viagens['Cargo'].fillna("NÃO IDENTIFICADO")
df_viagens

,Identificador do processo de viagem,Número da Proposta (PCDP),Situação,Viagem Urgente,Justificativa Urgência Viagem,Código do órgão superior,Nome do órgão superior,Código órgão solicitante,Nome órgão solicitante,CPF viajante,Nome,Cargo,Função,Descrição Função,Período - Data de início,Período - Data de fim,Destinos,Motivo,Valor diárias,Valor passagens,Valor devolução,Valor outros gastos,Despesas
0,17821923,000001/23-1C,Realizada,NÃO,Sem informação,26000,Ministério da Educação,26352,Fundação Universidade Federal do ABC,***.875.238-**,PRISCILA LEAL DA SILVA,PROFESSOR DO MAGISTERIO SUPERIOR,-1,Sem informação,01/01/2023,02/12/2023,Loughborough/Reino Unido,Intercâmbio acadêmico PCDP anterior 07/21-3C,0.00,0.00,0.00,0.00,0.00
1,18236583,000018/23,Realizada,NÃO,Sem informação,26000,Ministério da Educação,26236,Universidade Federal Fluminense,NaN,CAMILLA DUARTE DA SILVA,BIBLIOTECARIO-DOCUMENTALISTA,-1,Sem informação,16/01/2023,28/02/2024,Rio de Janeiro/RJ,Afastamento para pós-graduação stricto sensu n...,0.00,0.00,0.00,0.00,0.00
2,18288418,000007/23-1C,Realizada,SIM,Por necessidade do serviço.,52000,Ministério da Defesa,52121,Comando do Exército,***.621.358-**,ANDRE LUIS COSTA PITANGUEIRA,NÃO IDENTIFICADO,OfSuperior,OfSuperior,10/01/2023,29/01/2023,Nova York/Estados Unidos da América,Participar da revisão do COE Manual (Contingen...,39565.70,7434.13,0.00,1095.35,48095.18
3,18296348,000070/23,Realizada,SIM,A efetivação do empenho no Siafi e a respectiv...,26000,Ministério da Educação,26406,Instituto Federal do Espírito Santo,***.407.547-**,LIDIANY MIRANDA FERRAZ NUNES,AUXILIAR DE BIBLIIOTECA,-1,Sem informação,05/02/2023,10/02/2023,Vitória/ES,Participação das aulas de Mestrado Interinstit...,1013.96,0.00,0.00,0.00,1013.96
4,18302983,000001/23,Realizada,NÃO,Sem informação,52000,Ministério da Defesa,52111,Comando da Aeronáutica,***.855.388-**,GREGOR GASPAR,NÃO IDENTIFICADO,OfIntermed,OfIntermed,01/01/2023,07/07/2023,Gotemburgo/Suécia,PARTICIPAR DO CURSO DE PILOTO DE GRIPEN C/D.,0.00,14903.10,0.00,0.00,14903.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
826984,2023001618,Sem informaçã,Realizada,NÃO,Sem informação,-1,Sem informação,-1,Sem informação,***.748.269-**,BIANCA VIANA KIVEL,ANALISTA DO BANCO CENTRAL,-1,Sem informação,05/12/2023,06/12/2023,Sem informação,Participar de tratativas nos locais das reuniõ...,606.89,3331.29,0.00,0.00,3938.18
826985,2023001619,Sem informaçã,Realizada,NÃO,Sem informação,-1,Sem informação,-1,Sem informação,***.231.806-**,LUCAS ALVES FREIRE,PROCURADOR DO BANCO CENTRAL,-1,Sem informação,11/12/2023,13/12/2023,Sem informação,Participar da reunião do COPOM .,1089.00,1837.18,0.00,0.00,2926.18
826986,2023001628,Sem informaçã,Realizada,NÃO,Sem informação,-1,Sem informação,-1,Sem informação,***.632.011-**,SERGIO BOTELHO,TECNICO DO BANCO CENTRAL,-1,Sem informação,07/12/2023,08/12/2023,Sem informação,Acompanhamento do Presidente do BC - São Paulo,734.13,4555.95,0.00,0.00,5290.08
826987,2023001631,Sem informaçã,Realizada,NÃO,Sem informação,-1,Sem informação,-1,Sem informação,***.407.445-**,NELSON CAMPOS REIS JUNIOR,TECNICO DO BANCO CENTRAL,-1,Sem informação,11/12/2023,11/12/2023,Sem informação,Acompanhamento do Presidente - São Paulo.,319.28,3745.53,0.00,400.00,4464.81
